In [5]:
import torch
import torch.nn as nn

In [3]:
class DSC_Block(nn.Module):
    """
    A single Depthwise Separable Convolutional Block.
    Consists of:
      1. Depthwise Conv (Spatial/Temporal features)
      2. BatchNorm + ReLU
      3. Pointwise Conv (Channel mixing)
      4. BatchNorm + ReLU
      5. MaxPool (Optional, to reduce dimension)
    """
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(DSC_Block, self).__init__()
        
        # 1. Depthwise Convolution
        # groups=in_channels forces each channel to be convolved independently
        self.depthwise = nn.Conv1d(
            in_channels=in_channels,
            out_channels=in_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            groups=in_channels, 
            bias=False 
        )
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.relu1 = nn.ReLU()

        # 2. Pointwise Convolution
        # Kernel size is always 1. This mixes information across channels.
        self.pointwise = nn.Conv1d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.depthwise(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.pointwise(x)
        x = self.bn2(x)
        x = self.relu2(x)
        return x

class SisFallIoTNet(nn.Module):
    def __init__(self, num_channels=6, num_classes=2, window_size=200):
        super(SisFallIoTNet, self).__init__()
        
        # Initial standard convolution to expand features slightly
        # (Standard conv is okay here because input channels are low)
        self.initial_conv = nn.Sequential(
            nn.Conv1d(num_channels, 32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2) # Reduces time dim from 200 -> 100
        )

        # Stack of Separable Blocks
        # Increasing channels, decreasing time dimension
        self.layer1 = DSC_Block(32, 64, kernel_size=3)
        self.pool1 = nn.MaxPool1d(2) # 100 -> 50

        self.layer2 = DSC_Block(64, 128, kernel_size=3)
        self.pool2 = nn.MaxPool1d(2) # 50 -> 25
        
        self.layer3 = DSC_Block(128, 256, kernel_size=3)
        # No pool here, let Global Average Pooling handle the rest

        # Global Average Pooling
        # Averages the remaining time steps into a single vector
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)

        # Classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2), # Regularization for small datasets
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        # Input shape: [Batch, Channels, Time]
        
        x = self.initial_conv(x)
        
        x = self.pool1(self.layer1(x))
        x = self.pool2(self.layer2(x))
        x = self.layer3(x)
        
        # GAP reduces [Batch, 256, Time] -> [Batch, 256, 1]
        x = self.global_avg_pool(x)
        
        # Flatten for linear layer: [Batch, 256]
        x = x.view(x.size(0), -1)
        
        out = self.classifier(x)
        return out

# --- Verification ---

# Create dummy input: 
# Batch Size = 8
# Channels = 6 (Accel X/Y/Z + Gyro X/Y/Z)
# Window Length = 200 (e.g., 4 seconds @ 50Hz)
dummy_input = torch.randn(8, 3, 200)

# Initialize Model (Binary classification: Fall vs No Fall)
model = SisFallIoTNet(num_channels=3, num_classes=2)

# Check parameters count
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Trainable Parameters: {total_params}")

# Test Forward Pass
output = model(dummy_input)
print(f"Output Shape: {output.shape}") # Should be [8, 2]

Total Trainable Parameters: 45890
Output Shape: torch.Size([8, 2])
